In [2]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import speech_recognition as sr
import pyttsx3
import requests
import smtplib
import wikipedia
import threading
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime
import json
import wolframalpha

class VoiceAssistantGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Smart Voice Assistant")
        self.root.geometry("800x600")
    
        self.engine = pyttsx3.init()
        self.recognizer = sr.Recognizer()
        self.wolframalpha_client = wolframalpha.Client('ERHVG6-VY4A493TXW')
        
        self.is_listening = False
        self.command_var = tk.StringVar()
        self.status_var = tk.StringVar(value="Ready")
        
        self.create_gui()
        
    def create_gui(self):

        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)

        status_frame = ttk.Frame(main_frame)
        status_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Label(status_frame, textvariable=self.status_var).pack(side=tk.LEFT)
        
       
        self.output_display = scrolledtext.ScrolledText(
            main_frame,
            wrap=tk.WORD,
            height=20
        )
        self.output_display.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
       
        controls_frame = ttk.Frame(main_frame)
        controls_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(
            controls_frame,
            text="Start Listening",
            command=self.toggle_listening
        ).pack(side=tk.LEFT, padx=5)
        
        ttk.Button(
            controls_frame,
            text="Clear Display",
            command=self.clear_display
        ).pack(side=tk.LEFT, padx=5)
        
       
        features_frame = ttk.LabelFrame(main_frame, text="Quick Actions", padding="5")
        features_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(
            features_frame,
            text="Weather",
            command=lambda: self.handle_command("weather")
        ).pack(side=tk.LEFT, padx=5)
        
        ttk.Button(
            features_frame,
            text="Send Email",
            command=lambda: self.handle_command("send email")
        ).pack(side=tk.LEFT, padx=5)
        
        ttk.Button(
            features_frame,
            text="Ask Question",
            command=lambda: self.handle_command("question")
        ).pack(side=tk.LEFT, padx=5)
        
        ttk.Button(
            features_frame,
            text="Set Reminder",
            command=lambda: self.handle_command("reminder")
        ).pack(side=tk.LEFT, padx=5)

    def log_output(self, message, prefix=""):
        timestamp = datetime.now().strftime("%H:%M:%S")
        self.output_display.insert(tk.END, f"[{timestamp}] {prefix}{message}\n")
        self.output_display.see(tk.END)

    def speak(self, text):
        """Converts text to speech."""
        self.log_output(text, "Assistant: ")
        self.engine.say(text)
        self.engine.runAndWait()

    def listen(self):
        """Captures audio input and converts it to text."""
        with sr.Microphone() as source:
            self.status_var.set("Listening...")
            self.log_output("Listening...")
            
            try:
                audio = self.recognizer.listen(source, timeout=20)
                command = self.recognizer.recognize_google(audio)
                self.log_output(f"You said: {command}")
                return command.lower()
            except sr.UnknownValueError:
                self.speak("Sorry, I didn't catch that. Please say it again.")
                return None
            except sr.RequestError:
                self.speak("Network issue. Please check your connection.")
                return None


    def fetch_weather(self, city):
        """Fetches weather data for a given city."""
        api_key = "d494c6744fc62dc97eefad0d7c6dbea3"
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
        
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                weather = data['weather'][0]['description']
                temp = data['main']['temp']
                return f"The current weather in {city} is {weather} with a temperature of {temp}°C."
            else:
                return "Sorry, I couldn't fetch the weather information."
        except Exception as e:
            return f"Error fetching weather: {str(e)}"

    def send_email(self, receiver_email, subject, body):
        """Sends an email using SMTP."""
        sender_email = "YOUR_EMAIL"
        password = "YOUR_PASSWORD"
        
        try:
            msg = MIMEMultipart()
            msg['From'] = sender_email
            msg['To'] = receiver_email
            msg['Subject'] = subject
            msg.attach(MIMEText(body, 'plain'))
            
            with smtplib.SMTP('smtp.gmail.com', 587) as server:
                server.starttls()
                server.login(sender_email, password)
                server.send_message(msg)
            
            return "Email sent successfully!"
        except Exception as e:
            return f"Failed to send email. Error: {str(e)}"

    def get_knowledge_answer(self, question):
        """Attempts to answer general knowledge questions."""
        try:
            
            wolfram_result = self.wolframalpha_client.query(question)
            for pod in wolfram_result.pods:
                if pod.title in ['Result', 'Definition', 'Basic information']:
                    return pod.text
            
            
            wiki_result = wikipedia.summary(question, sentences=2)
            return wiki_result
            
        except Exception:
            try:
               
                wiki_result = wikipedia.summary(question, sentences=2)
                return wiki_result
            except Exception as e:
                return f"I'm sorry, I couldn't find an answer to that question. Error: {str(e)}"

    def handle_command(self, command):
        """Processes the user's command and performs actions."""
        if "weather" in command:
            self.speak("Which city should I check for the weather?")
            city = self.listen()
            if city:
                weather_info = self.fetch_weather(city)
                self.speak(weather_info)
                
        elif "send email" in command:
            self.speak("Please provide the recipient's email address.")
            receiver = self.listen()
            if receiver:
                self.speak("What is the subject of the email?")
                subject = self.listen()
                self.speak("What should the email say?")
                body = self.listen()
                if subject and body:
                    response = self.send_email(receiver, subject, body)
                    self.speak(response)
                    
        elif "question" in command:
            self.speak("What would you like to know?")
            question = self.listen()
            if question:
                answer = self.get_knowledge_answer(question)
                self.speak(answer)
                
        elif "reminder" in command:
            self.speak("What should I remind you about?")
            reminder = self.listen()
            if reminder:
                self.speak(f"I'll remind you to {reminder}.")
                
        elif "stop" in command:
            self.speak("Goodbye!")
            self.root.quit()
            
        else:
            self.speak("I'm sorry, I didn't understand that command.")

    def toggle_listening(self):
        """Toggles the listening state."""
        if not self.is_listening:
            self.is_listening = True
            threading.Thread(target=self.listening_loop, daemon=True).start()
        else:
            self.is_listening = False

    def listening_loop(self):
        """Continuous listening loop."""
        while self.is_listening:
            command = self.listen()
            if command:
                self.handle_command(command)
        self.status_var.set("Ready")

    def clear_display(self):
        """Clears the output display."""
        self.output_display.delete(1.0, tk.END)

def main():
    root = tk.Tk()
    app = VoiceAssistantGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

Exception ignored in: <function Variable.__del__ at 0x000001232B6E0AE0>
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\Lib\tkinter\__init__.py", line 410, in __del__
    if self._tk.getboolean(self._tk.call("info", "exists", self._name)):
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: main thread is not in main loop
Exception ignored in: <function Variable.__del__ at 0x000001232B6E0AE0>
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\Lib\tkinter\__init__.py", line 410, in __del__
    if self._tk.getboolean(self._tk.call("info", "exists", self._name)):
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: main thread is not in main loop


: 